In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 20s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 236s 5ms/step - loss: 1.5437 - acc: 0.4258 - val_loss: 2.6008 - val_acc: 0.2986
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:05 - loss: 1.1796 - acc: 0.5706

50000/50000 [==============================] - 219s 4ms/step - loss: 1.1101 - acc: 0.5980 - val_loss: 2.1690 - val_acc: 0.4351
Epoch 3/50
45568/50000 [==========================>...] - ETA: 18s - loss: 0.9515 - acc: 0.6592

50000/50000 [==============================] - 219s 4ms/step - loss: 0.9472 - acc: 0.6614 - val_loss: 1.2448 - val_acc: 0.5955
Epoch 4/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.8603 - acc: 0.6928 - val_loss: 1.3250 - val_acc: 0.5755
Epoch 5/50
 3328/50000 [>.............................] - ETA: 3:10 - loss: 0.8068 - acc: 0.7052

50000/50000 [==============================] - 217s 4ms/step - loss: 0.7962 - acc: 0.7164 - val_loss: 1.0242 - val_acc: 0.6637
Epoch 6/50
39424/50000 [======================>.......] - ETA: 42s - loss: 0.7393 - acc: 0.7371

50000/50000 [==============================] - 218s 4ms/step - loss: 0.7376 - acc: 0.7374 - val_loss: 1.2466 - val_acc: 0.6302
Epoch 7/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.6909 - acc: 0.7560 - val_loss: 1.0225 - val_acc: 0.6837
Epoch 8/50
 1792/50000 [>.............................] - ETA: 3:16 - loss: 0.6657 - acc: 0.7567

...

50000/50000 [==============================] - 218s 4ms/step - loss: 0.2293 - acc: 0.9173 - val_loss: 0.8225 - val_acc: 0.8017
Epoch 45/50
38656/50000 [======================>.......] - ETA: 46s - loss: 0.2346 - acc: 0.9181

50000/50000 [==============================] - 218s 4ms/step - loss: 0.2329 - acc: 0.9178 - val_loss: 0.7228 - val_acc: 0.8252
Epoch 46/50
50000/50000 [==============================] - 219s 4ms/step - loss: 0.2293 - acc: 0.9180 - val_loss: 0.9828 - val_acc: 0.7838
Epoch 47/50
 1536/50000 [..............................] - ETA: 3:16 - loss: 0.2078 - acc: 0.9277

50000/50000 [==============================] - 218s 4ms/step - loss: 0.2253 - acc: 0.9190 - val_loss: 0.6636 - val_acc: 0.8366
Epoch 48/50
38656/50000 [======================>.......] - ETA: 45s - loss: 0.2145 - acc: 0.9220

50000/50000 [==============================] - 218s 4ms/step - loss: 0.2165 - acc: 0.9221 - val_loss: 0.8460 - val_acc: 0.7996
Epoch 49/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.2215 - acc: 0.9213 - val_loss: 0.8082 - val_acc: 0.8050
Epoch 50/50
 1536/50000 [..............................] - ETA: 3:17 - loss: 0.2227 - acc: 0.9212

50000/50000 [==============================] - 218s 4ms/step - loss: 0.2094 - acc: 0.9244 - val_loss: 0.7873 - val_acc: 0.8111


In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 21s 2ms/step
Test loss: 0.7872767700910568
Test accuracy: 0.8111


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')